In [2]:
%%capture
!pip install transformers datasets tabulate

In [3]:
import numpy as np
import torch
import torch.nn.functional as F
import torch.nn as nn
import math
from torch.utils.data import DataLoader
from tabulate import tabulate
from datasets import load_dataset

from tqdm.notebook import tqdm
from transformers import BertTokenizer
torch.cuda.is_available()

True

This is a template of the notebook that you should complete and enrich with your own code.

First cells will be the same than the ones of the lab on text convolution.

# Data loading


In [4]:
dataset = load_dataset("scikit-learn/imdb", split="train")
print(dataset)

Dataset({
    features: ['review', 'sentiment'],
    num_rows: 50000
})


# Pre-processing / Tokenization

This is a very important step. It maybe boring but very important. In this session we will be lazy, but in real life, the time spent on inspecting and cleaning data is never wasted. It is true for text, but also for everything.



In PyTorch, everything is tensor. Words are replaced by indices. A sentence, is therefore a sequence of indices (long integers). In the first HW, you constructed a `WhiteSpaceTokenizer`. Here we will use an already built tokenizer. It is more appropriate to transformers. It relies on sub-word units, and converts everything in lower case. This is not always the best choice, but here it will be sufficient. To quote the documentation, this tokenizer allows you to:
- Tokenize (splitting strings in sub-word token strings), converttokens strings to ids and back, and encoding/decoding (i.e., tokenizing and converting to integers).
- Add new tokens to the vocabulary in a way that is independent of the underlying structure (BPE, SentencePiece…).
- Manage special tokens (like mask, beginning-of-sentence, etc.): adding them, assigning them to attributes in the tokenizer for easy access and making sure they are not split during tokenization.

Here we are going to use the tokenizer from the well known Bert model, that we can directly download.

In [5]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", do_lower_case=True)


In [6]:
def preprocessing_fn(x, tokenizer):
    x["review_ids"] = tokenizer(
        x["review"],
        add_special_tokens=False,
        truncation=True,
        max_length=256,
        padding=False,
        return_attention_mask=False,
    )["input_ids"]
    x["label"] = 0 if x["sentiment"] == "negative" else 1
    return x


In [7]:
tokenizer(
        "[UNK] ",
        add_special_tokens=False,
        truncation=True,
        max_length=256,
        padding=False,
        return_attention_mask=False,
    )["input_ids"]

[100]

Same celel than in the lab session.

🚧 **TODO** 🚧

Read the documentation about HuggingFace dataset and complete the code below.
You should:
- Shuffle the dataset
- For computational reasons, use only a total of **5000 samples**.
- Tokenize the dataset with the `preprocessing_fn`. (*Hint: use the `Dataset.map` method from HuggingFace*).
- Keep only columns `review_ids` and `label`.
- Make a train/validation split, (**80% / 20%**). Call these dataset `train_set` and `valid_set`.


## Q1 - Tokenize and Split Data

In [8]:
n_samples = 50000  # the number of training example

# We first shuffle the data !
data_shuffled = dataset.shuffle()

# Select 5000 samples
data_shuffled_sampled = data_shuffled.select(range(n_samples))

# Tokenize the dataset
data_tokenized = data_shuffled_sampled.map(lambda x: preprocessing_fn(x, tokenizer))

# Remove useless columns
data_tokenized = data_tokenized.remove_columns(["review", "sentiment"])

# Split the train and validation
split = data_tokenized.train_test_split(test_size=0.2)
train_set = split['train']
valid_set = split["test"]

document_train_set = train_set["review_ids"]
document_valid_set = valid_set["review_ids"]


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

## Q2 -  Make Positive context with padding

In [9]:
def extract_words_contexts(document, radius, pad_token_id=0):
    words = []
    contexts = []
    
    for i in range(len(document)):
        words.append(document[i])
        
        # Create context
        context = []
        for j in range(i - radius, i + radius + 1):
            if j != i:  # Exclude the word itself
                if 0 <= j < len(document):
                    context.append(document[j])
                else:
                    # Handling borders: pad with pad_token_id (usually 0 for BERT)
                    context.append(pad_token_id)
        
        contexts.append(context)
    
    return words, contexts

## Q3 Apply to dataset

In [10]:
def flatten_dataset_to_list(dataset, R):
    all_ids = []
    all_contexts = []

    for document in tqdm(dataset):
        ids, contexts = extract_words_contexts(document, R)
        all_ids.extend(ids)
        all_contexts.extend(contexts)

    return all_ids, all_contexts

In [11]:
data_tokenized_flattened = flatten_dataset_to_list(data_tokenized["review_ids"], 30)

  0%|          | 0/10000 [00:00<?, ?it/s]

# Q4

In [12]:
R,K = 10,5

In [13]:
train_ids, train_context = flatten_dataset_to_list(document_train_set, R)
valid_ids, valid_context = flatten_dataset_to_list(document_valid_set, R)

  0%|          | 0/8000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

# Q5 - Context Dataset

In [14]:
from torch.utils.data import Dataset

class ContextDataset(Dataset):
    def __init__(self, words, contexts):
      self.words = words
      self.contexts = contexts

    def __len__(self):
        return len(self.words)

    def __getitem__(self, idx: int):
        return self.words[idx], self.contexts[idx]

In [15]:
train_set = ContextDataset(train_ids, train_context)
valid_set = ContextDataset(valid_ids, valid_context)

# Q6 - Negative Context

In [16]:
def collate_fn(batch, vocab_size, K):
    word_ids = []
    positive_context_ids = []
    negative_context_ids = []

    for word_id, pos_context in batch:
        word_ids.append(word_id)
        positive_context_ids.append(pos_context)
        
        # Generate negative context by random sampling
        neg_context_size = 2 * len(pos_context) * K
        neg_context = np.random.randint(0, vocab_size, size=neg_context_size).tolist()
        negative_context_ids.append(neg_context)

    return {
        'word_id': torch.tensor(word_ids, dtype=torch.long),
        'positive_context_ids': torch.tensor(positive_context_ids, dtype=torch.long),
        'negative_context_ids': torch.tensor(negative_context_ids, dtype=torch.long)
    }

vocab_size = tokenizer.vocab_size

# Q7

In [17]:
train_loader = DataLoader(train_set, batch_size=32, shuffle=True, collate_fn=lambda x: collate_fn(x, vocab_size=vocab_size, K=K))
valid_loader = DataLoader(valid_set, batch_size=32, shuffle=False, collate_fn=lambda x: collate_fn(x, vocab_size=vocab_size, K=K))

# Q8

In [18]:
batch = next(iter(train_loader))
batch

{'word_id': tensor([ 2003,  2031,  2471,  2009,  1010,  2143,  1012,  2147,  3510,  2002,
          2135,  2022,  1998,  1010,  8840,  2323,  2228,  2682,  2040,  3993,
          1012,  1044,  1012,  1037,  1056,  2031,  3849, 15338,  2143,  1005,
         17567,  4931]),
 'positive_context_ids': tensor([[ 5886, 10127,  1029,  6684,  2151,  1012,  2021,  7804, 20296,  2121,
           3376,  1012,     0,     0,     0,     0,     0,     0,     0,     0],
         [ 2044,  1037,  2096,  2017,  1005,  2222,  2593,  2025,  2729,  2030,
           5357,  6680,  1012,  2012,  2151,  3446,  1010,  2023,  2038,  7564],
         [ 1010,  1006,  2011,  2008,  1045,  2812,  2008,  2027,  1005,  2128,
           1996,  2168,  3924,  1010,  2029,  2003,  1037,  2978,  1997,  1037],
         [ 2016,  3248,  2117, 15888,  2000,  2877,  2158,  4702, 21213,  1012,
           1005,  1055,  1037, 12063,  1010,  2138,  2016,  2038,  2062,  2895],
         [ 2019,  9643,  2135, 11771,  1010, 15703,  1998, 

In [19]:
batch['word_id'].shape

torch.Size([32])

In [20]:
batch['positive_context_ids'].shape

torch.Size([32, 20])

In [21]:
batch['negative_context_ids'].shape

torch.Size([32, 200])

# Q9 - Word2Vec Model



In [22]:
class Word2Vec(nn.Module):
    def __init__(self, vocab_size, embedding_dim,device = 'cuda'):
        super(Word2Vec, self).__init__()
        self.device = device
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim).to(device)
        self.context_embeddings = nn.Embedding(vocab_size, embedding_dim).to(device)

    def forward(self, word_id, pos_context_ids, neg_context_ids):
        word_embed = self.word_embeddings(word_id)
        #print("word",word_embed.shape)

        pos_context_embed = self.context_embeddings(pos_context_ids).to(self.device)
        #print("pos",pos_context_embed.shape)
        neg_context_embed = self.context_embeddings(neg_context_ids).to(self.device)
        #print("neg",neg_context_embed.shape)

        pos_dot_product = torch.bmm(pos_context_embed, word_embed.unsqueeze(2)).squeeze(2)
        #print("pos_dot_product",pos_dot_product.shape)

        neg_dot_product = torch.bmm(neg_context_embed, word_embed.unsqueeze(2)).squeeze(2)
        #print("neg_dot_product",neg_dot_product.shape)

        pos_similarity = torch.sigmoid(pos_dot_product)
        #print("pos_similarity",pos_similarity.shape)  # (batch_size, 2R)

        neg_similarity = torch.sigmoid(neg_dot_product)
        #print("neg_similarity",neg_similarity.shape) # (batch_size, 2R * K)

        pos_loss = -torch.log(pos_similarity + 1e-8).sum(1)  # Sum over all positive contexts
        neg_loss = -torch.log(1 - neg_similarity + 1e-8).sum(1)  # Sum over all negative contexts
        loss = pos_loss + neg_loss
        #print(loss.shape)

        return loss.mean()

# Q10

In [23]:
len(train_loader)

51403

In [24]:
def train_model(model, batch_size, epochs, lr =0.001):
    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, 
        collate_fn=lambda x: collate_fn(x, vocab_size=vocab_size, K=K))
    
    valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=False,
        collate_fn=lambda x: collate_fn(x, vocab_size=vocab_size, K=K))
    
    print(len(train_loader))
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    device = 'cuda'
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        total_loss_val = 0

        print("### Train")

        for batch in tqdm(train_loader):

            word_id = batch['word_id'].to(device)
            positive_context_ids = batch['positive_context_ids'].to(device)
            negative_context_ids = batch['negative_context_ids'].to(device)

            optimizer.zero_grad()
            loss = model(word_id, positive_context_ids, negative_context_ids)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        print("### Val")
        with torch.no_grad():
            for batch in tqdm(valid_loader):
                word_id = batch['word_id'].to(device)
                positive_context_ids = batch['positive_context_ids'].to(device)
                negative_context_ids = batch['negative_context_ids'].to(device)
                loss = model(word_id, positive_context_ids, negative_context_ids)
                total_loss_val += loss.item()
        
        avg_loss = total_loss / len(train_loader)
        avg_loss_val = total_loss_val / len(valid_loader)

        print(f'Epoch [{epoch + 1}/{epochs}], Loss: {avg_loss:.4f}, Val_Loss: {avg_loss_val:.4f}')


In [ ]:
# Define hyperparameters
vocab_size = vocab_size  # Size of the vocabulary
embedding_dim = 100  # Dimension of the word embeddings
batch_size = 1024  # Batch size (B)
epochs = 20  # Number of epochs (E)

# Create the Word2Vec model
model = Word2Vec(vocab_size, embedding_dim)

# Assuming `train_dataset` is already prepared
# Train the model
train_model(model,batch_size, epochs)


# Q11 - Cosine Similarity Validation

In [27]:
import torch.nn.functional as F

def validate_word2vec(model, test_loader, R, K, B): # updated here
    
    # test_loader = DataLoader(test_dataset, batch_size=B, shuffle=False, collate_fn=lambda batch: collate_fn(batch, R, K)) # removed this line

    model.eval()  # Set the model to evaluation mode

    total_pos_sim = 0
    total_neg_sim = 0
    pos_count = 0
    neg_count = 0

    with torch.no_grad():  # Disable gradient computation for validation
        for batch in test_loader:
            word_id = batch['word_id'].to('cuda')
            positive_context_ids = batch['positive_context_ids'].to('cuda')
            negative_context_ids = batch['negative_context_ids'].to('cuda')

            # Get embeddings for words, positive contexts, and negative contexts
            word_embeddings = model.word_embeddings(word_id)
            positive_embeddings = model.context_embeddings(positive_context_ids)
            negative_embeddings = model.context_embeddings(negative_context_ids)

            # Cosine similarity for positive contexts
            pos_similarity = F.cosine_similarity(word_embeddings.unsqueeze(1), positive_embeddings, dim=-1)
            total_pos_sim += pos_similarity.sum().item()
            pos_count += pos_similarity.numel()

            # Cosine similarity for negative contexts
            neg_similarity = F.cosine_similarity(word_embeddings.unsqueeze(1), negative_embeddings, dim=-1)
            total_neg_sim += neg_similarity.sum().item()
            neg_count += neg_similarity.numel()

    # Average cosine similarities
    avg_pos_sim = total_pos_sim / pos_count
    avg_neg_sim = total_neg_sim / neg_count

    print(f"Avg Positive Context Similarity: {avg_pos_sim:.4f}")
    print(f"Avg Negative Context Similarity: {avg_neg_sim:.4f}")

    return avg_pos_sim, avg_neg_sim

In [28]:
avg_pos_sim, avg_neg_sim = validate_word2vec(model, valid_loader, R, K, batch_size)


Avg Positive Context Similarity: 0.2980
Avg Negative Context Similarity: -0.3560


# Q12 - Save

In [29]:
torch.save(model.state_dict(), f"model2_dim-{embedding_dim}_radius-{R}_ratio-{K}_batch-{batch_size}_epoch-{epochs}_samples-{n_samples}.ckpt")

In [30]:
model

Word2Vec(
  (word_embeddings): Embedding(30522, 100)
  (context_embeddings): Embedding(30522, 100)
)